In [1]:
BUCKET = 'qwiklabs-gcp-03-223e89e2542f'
PROJECT = BUCKET
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.5'

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [21]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer_text_distributed
python -m trainer_text_distributed.task \
  --output_dir=text_trained \
  --train_data_path=gs://${BUCKET}/data_text/train.csv \
  --test_data_path=gs://${BUCKET}/data_text/test.csv \
  --epochs=2

Loading training/validation data ...
Training the model ...
Model: "nnlm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None,)]                 0         
_________________________________________________________________
input (KerasLayer)           (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
output (Dense)               (None, 3)                 99        
Total params: 124,653,123
Trainable params: 10,435
Non-trainable params: 124,642,688
_________________________________________________________________
None
Epoch 1/2
3125/3125 [=================

2022-02-23 12:39:33.713369: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-02-23 12:39:33.780721: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-23 12:39:35.060194: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_349"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 

In [25]:
%%bash
OUTDIR=gs://${BUCKET}/text_trained_model
INPUTDIR=gs://${BUCKET}/data_text/train.csv
JOBNAME=text_model_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer_text_distributed.task \
  --package-path=$(pwd)/trainer_text_distributed \
  --job-dir=gs://${BUCKET}/text_trained_model \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=basic-gpu \
  --runtime-version=2.5 \
  --python-version=3.7 \
  -- \
  --output_dir=${OUTDIR} \
  --train_data_path=${INPUTDIR} \
  --test_data_path=gs://${BUCKET}/data_text/test.csv \
  --epochs=50

jobId: text_model_220223_124528
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [text_model_220223_124528] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe text_model_220223_124528

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs text_model_220223_124528


In [31]:
%%bash
gcloud ai-platform jobs describe text_model_220223_124528

createTime: '2022-02-23T12:45:32Z'
endTime: '2022-02-23T13:13:46Z'
etag: IAqL-97LevU=
jobId: text_model_220223_124528
jobPosition: '0'
startTime: '2022-02-23T12:52:42Z'
state: SUCCEEDED
trainingInput:
  args:
  - --output_dir=gs://qwiklabs-gcp-03-223e89e2542f/text_trained_model
  - --train_data_path=gs://qwiklabs-gcp-03-223e89e2542f/data_text/train.csv
  - --test_data_path=gs://qwiklabs-gcp-03-223e89e2542f/data_text/test.csv
  - --epochs=50
  jobDir: gs://qwiklabs-gcp-03-223e89e2542f/text_trained_model
  packageUris:
  - gs://qwiklabs-gcp-03-223e89e2542f/text_model_220223_124528/feedf372c5e57128f70a97019b267eaef3d8c67644fb634bb5f2325e07fd9e21/trainer_text_distributed-0.0.0.tar.gz
  pythonModule: trainer_text_distributed.task
  pythonVersion: '3.7'
  region: us-central1
  runtimeVersion: '2.5'
  scaleTier: BASIC_GPU
trainingOutput:
  consumedMLUnits: 0.52



View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/text_model_220223_124528?project=qwiklabs-gcp-03-223e89e2542f

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id%2Ftext_model_220223_124528&project=qwiklabs-gcp-03-223e89e2542f


In [49]:
%%bash
MODEL_NAME="text_model_distributed"
MODEL_VERSION="text_ml_on_gcp"
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/text_trained_model/1* | tail -1 | tr -d '[:space:]')
gcloud ai-platform models create ${MODEL_NAME}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.5 \
    --python-version=3.7

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/qwiklabs-gcp-03-223e89e2542f/models/text_model_distributed].
Using endpoint [https://us-central1-ml.googleapis.com/]
Creating version (this might take a few minutes)......
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [51]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "text_model_distributed"
MODEL_VERSION = "text_ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
print(token)

ya29.c.b0AXv0zTPgzTjcx-TqSgHO39dNwG601Ts6KhMnpvK84SVB5TrhXrcMZx42ghu2Ddo7kw80p5tLXMzfZBqjdGe19z_mkzXcuaTIT9UU0LaBLwOwlV3Ci95QXNbKpBLyqRk34iwVB5WE-yNd3NWRH_x_FkjqVH-MKJxcB6Yy0tguixalNEsjtnfYfISXoZsRTgDym_bg0LdhlXs....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [55]:
api = "https://us-central1-ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict" \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {"Authorization": "Bearer " + token }

model_input="very bad canned food"

data = {"instances": [model_input]}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n    "predictions": [[0.911958098, 0.0130848847, 0.0749569833]\n    ]\n}'
